# Projet numérique : Câble sous-marin

*Auteurs : Valentin Fenoux et Mathis Boutrouelle*

## Questions théoriques

1. On se sert de la méthode de Monte-Carlo pour approximer la valeur de l'espérance conditionnelle. On suppose que les simulations sont une suite de variables aléatoires indépendantes $(X_n)_{n \in \mathbb{N}^*}$ de même loi et intégrable. Alors : $M_n= \frac{X_1+...+X_n}{n}$ converge presque sûrement vers $E^*$ d'après la loi des grands nombres.

2. On pose le vecteur $\bar{Z} = (Z,Z') = (Z(x_0),...,Z(x_N),Z(x_{j_1}),...,Z(x_{j_n}))$ qui est un vecteur aléatoire gaussien de moyenne $m = (µ,...,µ)$ et de matrice de covariance $\Sigma$ définie positive. Alors d'après le cours Probabilités IV, la densité conditionnelle de Z sachant les profondeurs aux points d'observation est : 
$$f_{Z|Z(x_{j_1})=z(x_{j_1}),...,Z(x_{j_n})=z(x_{j_n})}(y) = \frac{1}{(2\pi)^{\frac{N+1}{2}}\sqrt{\det (C S_Z)}}\exp \left(-\frac{1}{2}\left(y - µ)\right)^t C S_Z^{-1}\left(y - µ)\right)\right)$$
D'où la loi conditionnelle. Avec $C = \begin{pmatrix} \Sigma & C_{Z,Z'} \\ C_{Z',Z} & C_{Z'} \end{pmatrix} \:$  et  $\: CS_Z = \Sigma - C_{Z,Z'}C_{Z'}^{-1}C_{Z',Z}$

3. Comme $Y$ est gausssien, on a (p.11 poly Proba V) $Z$ qui est gaussien d'espérance $m$ et de matrice de covariance $C$. De plus comme les $Y_i$ sont des variables aléatoires gaussiennes centrées, réduites et indépendantes, $C = RR^t$

4. - Soient $(U_k,V_k)_{0\leqslant k\leqslant N} \;$ $2(N+1)$ variables aléatoires uniformes indépendantes dans $]0,1[$, simulées avec le module random de python.
   - Alors les $Y_k = \sqrt{-2ln(U_k)}cos(2\pi V_k)$ sont des variables aléatoires gaussiennes centrées réduites.
   - On a alors $Z = µ + RY$, avec R décomposition de Cholesky de $CS_Z$ pouvant être obtenue grâce à un algorithme, et $CS_Z$ grâce à la fonction décroissante C.

## Implémentation

### Préambule

In [2]:
# Chargement de dépendance

import numpy as np
import math
import matplotlib.pyplot as plt

In [3]:
# Discrétisation

A = 0
B = 500
N = 101
Delta = (B - A)/(N - 1)
discretization_indexes = np.arange(N)
discretization = discretization_indexes*Delta

In [4]:
# Paramètres du modèle

mu = -5
a = 50
sigma2 = 12

In [5]:
# Données

observation_indexes = [0, 20, 40, 60, 80, 100]
depth = np.array([0, -4, -12.8, -1, -6.5, 0])

In [10]:
# Indices des composantes correspondant aux observations et aux composantes non observées

unknown_indexes = list(set(discretization_indexes)-set(observation_indexes))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


### Questions

#### 1.

In [7]:
def cov(H) :
    n = len(H[0])
    C = np.zeros([n,n])
    for i in range(n) :
        for j in range(n) :
            C[i,j] = sigma2*np.exp(-H[i,j]/a)
            
    return C

#### 2. Matrice de distance

In [8]:
# Matrice de distance vide
D = np.zeros([N,N])
# Ajout des longueurs
for i,x in enumerate(discretization) :
    for j,y in enumerate(discretization) :
        D[i,j] = abs(x - y)   

#### 3. Matrice de covariance

In [9]:
Cov_Z = cov(D)